In [7]:
### Load packages
using LinearAlgebra, Statistics, Compat, HypothesisTests # Statistical analysis
using RDatasets, MultivariateStats # Principal Component Analysis
using Distributions, GLM # OLS
using DataFrames, CSV # Data analysis
using Dates, Plots #printf # Misc...

In [8]:
### Supportive function -- part I
function blkdig(vararg)
    stop_row = 0
    stop_col = 0
    column = 0
    row = 0
    for i in 1:length(vararg)
        column = column + size(vararg[i],2)
        row = row + size(vararg[i],1)
    end
    
    blk = zeros(row, column)
    
    for i in 1:length(vararg)
        mat_comp = vararg[i]
        for j in 1:size(mat_comp, 2)
            for k in 1:size(mat_comp, 1)
                blk[stop_row + k, stop_col + j] = mat_comp[k,j]
            end
        end
        stop_row = stop_row + size(mat_comp,1)
        stop_col = stop_col + size(mat_comp,2)
    end
    
    return blk
end

### Supportive function -- part II
blocks = [1 0 0 1 0
            1 0 1 0 0
            1 0 0 1 0
            1 1 1 1 1]
function sort_vec(blocks)
    uniq = blocks[1,:]'
    
    for i in 2:size(blocks,1)
        sum = 0
        for j in 1:size(uniq,1)
            sum = sum + (blocks[i,:]' == uniq[j,:]')
        end
        if sum == 0
            uniq = vcat(uniq, blocks[i,:]')
        end
    end
    return uniq
end

function ismember(blocks, str)
    idx = []
    for i in 1:size(blocks,1)
        if blocks[i,:] == str
            push!(idx, i)
        end
    end
    return idx
end

sort_vec(blocks)

3×5 Array{Int64,2}:
 1  0  0  1  0
 1  0  1  0  0
 1  1  1  1  1

In [9]:
struct Spec
    r::Int
    p::Int
    nQ::Int
    blocks::AbstractMatrix{Float64}
end

struct inits
    A::AbstractMatrix{Float64}
    G::AbstractMatrix{Float64}
    Q::AbstractMatrix{Float64}
    R::AbstractMatrix{Float64}
    Z::AbstractMatrix{Float64}
    V::AbstractMatrix{Float64}
end

mutable struct Sm
    Zm::AbstractMatrix{Float64}
    ZmU::AbstractMatrix{Float64}
    Vm::AbstractArray{Float64,3}
    VmU::AbstractArray{Float64,3}
    loglik::Float64
    k_t::AbstractMatrix{Float64}
end

mutable struct Sm1
    ZmT::AbstractMatrix{Float64}
    VmT::AbstractArray{Float64,3}
    VmT_1::AbstractArray{Float64,3}
end

In [83]:
##### Creating Dynamic Factor Model (DFM) to forecast GDP: 
# y_t - μ = G_θ * x_t
# x_{t+1} = A_θ * x_t + η_t
function dfm(X, Spec, inits, threshold = 1e-5)
    ### Set up
    #printf("Estimating the dynamic factor model (DFM) ... \n\n")
    
    (T, N) = size(X)
    # Identify spec 
    r = Spec.r
    p = Spec.p
    nQ = Spec.nQ
    blocks = Spec.blocks
    
    i_idio = Bool.([ones(N-nQ,1); zeros(nQ,1)])
    
    # We should not have constraint
    R_mat = [2 -1 0 0 0
             3 0 -1 0 0
             2 0 0 -1 0
             1 0 0 0 -1]
    q = zeros(4,1)
    
    #max_iter = 5000
    max_iter = 5
    
    ### Prepare data
    #Mx = mean(X, "omitnan")
    #Wx = std(X, "omitnan")
    #xMissing = (X - repmat(Mx, T, 1))./repmat(Wx, T,1) #? Standardize series???
    
    ### Initial Condition
    A = inits.A
    G = inits.G
    Q = inits.Q
    R = inits.R
    Z_0 = inits.Z
    V_0 = inits.V
    
    previous_loglik = -Inf
    num_iter = 0
    LL = -Inf
    converged = false
    
    # ? y for the estimation is WITH missing data
    
    ### EM loop
    #? remove the leading and ending nans
    # y_est
    println("run EM loop")
    while (num_iter < max_iter) & !converged
        println(num_iter+1)
        (G_new, R_new, A_new, Q_new, Z_0, V_0, loglik ) = EMstep(X, A, G, Q, R, Z_0, V_0, r, p, R_mat, q, nQ, i_idio, blocks)
        G = G_new
        R = R_new
        A = A_new
        Q = Q_new
        
        #if num_iter > 2 
            # change from decrease(num_iter+1)
        #    converged = em_converged(loglik, previous_loglik, threshold, check_decreased = 1)[1] #####?????
        #end
        
        #if (mod(num_iter,10) == 0) && (num_iter > 0)
            ##### Display something
        #end
        
        #LL = [LL loglik]
        #previous_loglik = loglik
        num_iter = num_iter + 1
        #println(LL)
        #println(converged)
    end
    println("finish EM loop")
    
    ##### Display something
    
    ### Final run of KF
    Zsmooth = runKF(X, A, G, Q, R, Z_0, V_0)[1] # use X with missing data
    x_sm = G*Zsmooth[:,2:end] # originally Zsm*G'
    
    ##### Display something
    
    return x_sm

end

dfm (generic function with 2 methods)

In [84]:
### Y_t = G_t X_t + e_t for e_t ~ N(0, R)
### X_t = A X_{t-1} + mu_t for mu_t ~ N(0,Q)
function EMstep(y,A,G,Q,R,x_0, σ_0, r, p, R_mat,q, nQ, i_idio, blocks)
    
    (n,T) = size(y)
    nM = n - nQ
    pC = size(R_mat,2)
    ppC = max(p, pC)
    num_blocks = size(blocks, 2) # how many columns are there?
    r = convert(Array{Int64}, ones(1,num_blocks))
    
    
    (x_sm, σ_sm, vv_sm, loglik) = runKF(y, A, G, Q, R, x_0, σ_0)
    A_new = A
    Q_new = Q
    σ_0_new = σ_0

    
    ##### Transition equation
    ### Update factor parameters individually
    b_subset = 1:ppC
    A_denom = zeros(ppC, ppC)
    A_nom = zeros(ppC, ppC)
    Q_update = zeros(ppC, ppC)
    
    for t in 1:T-1
        A_denom = A_denom + x_sm[b_subset, t+1]*x_sm[b_subset, t]' + vv_sm[b_subset, b_subset, t+1]
        A_nom = A_nom + x_sm[b_subset, t]*x_sm[b_subset, t]' + σ_sm[b_subset, b_subset, t]
    end
    
    A_i = Diagonal(A_denom*inv(A_nom))
    A_new[b_subset, b_subset] = A_i
    
    for t in 1:T-1
        Q_update = Q_update + x_sm[b_subset, t+1]*x_sm[b_subset, t+1]' + σ_sm[b_subset, b_subset, t+1]
        Q_update = Q_update - A_i*(x_sm[b_subset, t]*x_sm[b_subset, t+1]'+ vv_sm[b_subset, b_subset, t+1])
    end
    Q_i = Q_update / (T-1)
    Q_new[b_subset, b_subset] = Diagonal(Q_i)
    σ_0_new[b_subset, b_subset] = Diagonal(σ_sm[b_subset, b_subset, 1])
    
    #for i in 1:num_blocks
    #    r_i = r[i]
    #    rp = r_i*p
    #    if i == 1
    #        rp1 = 0
    #    elseif i == 2
    #        rp1 = r[1]*ppC
    #   else
    #        rp1 = sum(r[1:i-1]*ppC)
    #    end
        
    #    b_subset = rp1+1:rp1+rp # Subset blocks: Helps for subsetting x_sm, σ_sm
    #    t_start = rp1+1
    #    t_end = rp1+r_i*ppC

    #    EZZ = x_sm[b_subset, 2:end]*(x_sm[b_subset, 2:end])' + sum(σ_sm[b_subset, b_subset, 2:end], dims = 3)
    #    EZZ_BB = x_sm[b_subset, 1:end-1]*(x_sm[b_subset, 1:end-1])' + sum(σ_sm[b_subset, b_subset, 1:end-1], dims = 3)
    #    EZZ_FB = x_sm[b_subset, 2:end]*(x_sm[b_subset, 1:end-1])' + sum(vv_sm[b_subset, b_subset, :], dims = 3)

    #    A_i = A[t_start:t_end, t_start:t_end]
    #    Q_i = Q[t_start:t_end, t_start:t_end]
    #    A_i[1:r_i,1:rp] = EZZ_FB[1:r_i,1:rp]*inv(EZZ_BB[1:r_i,1:rp])
    #    Q_i[1:r_i,1:r_i] = (EZZ[1:r_i,1:r_i] - A_i[1:r_i,1:rp]*EZZ_FB[1:r_i,1:rp]') / T

    #    A_new[t_start:t_end, t_start:t_end] = A_i
    #    Q_new[t_start:t_end, t_start:t_end] = Q_i
    #    σ_0_new[t_start:t_end, t_start:t_end] = σ_sm[t_start:t_end, t_start:t_end, 1]
    #end
    
    ### Update parameters for idiosyncratic component
    rp1 = sum(r)*ppC #25
    niM = sum(i_idio[1:nM]) #118
    t_start = rp1+1
    i_subset = t_start:rp1+niM
    
    EZZ = Diagonal(Diagonal(x_sm[t_start:end,2:end]*(x_sm[t_start:end, 2:end])'))+Diagonal(Diagonal(sum(σ_sm[t_start:end, t_start:end, 2:end], dims = 3)[:,:,1]))
    EZZ_BB = Diagonal(Diagonal(x_sm[t_start:end,1:end-1]*(x_sm[t_start:end, 1:end-1])'))+Diagonal(Diagonal(sum(σ_sm[t_start:end, t_start:end, 1:end-1], dims = 3)[:,:,1]))
    EZZ_FB = Diagonal(Diagonal(x_sm[t_start:end,2:end]*(x_sm[t_start:end, 1:end-1])'))+Diagonal(Diagonal(sum(vv_sm[t_start:end, t_start:end, :], dims = 3)[:,:,1]))
    
    A_i = EZZ_FB*Diagonal(inv(Diagonal((EZZ_BB))))
    Q_i = (EZZ - A_i*EZZ_FB') / T
    
    A_new[i_subset, i_subset] = A_i[1:niM,1:niM]
    Q_new[i_subset, i_subset] = Q_i[1:niM,1:niM]
    σ_0_new[i_subset, i_subset] = Diagonal(Diagonal(σ_sm[i_subset,i_subset,1]))
    #println("Transition Equation DONE")
    
    ##### Observation equation
    ### Maximization step
    x_0 = x_sm[:,1]
    
    nanY = ismissing.(y)
    y[nanY] .= 0
    
    G_new = G
    
    bl = sort_vec(blocks) # list of unique blocks
    n_bl = size(bl,1) # number of unique blocks
    
    #bl_idxM = []
    #bl_idxQ = []
    #R_con = []
    #q_con = []
    
    #for i in 1:num_blocks
        #bl_idxQ = [bl_idxQ repeat(bl[:,i],1,r[i]*ppC)]
        #bl_idxM = [bl_idxM repeat(bl[:,i],1,r[i]) zeros(n_bl, r[i]*(ppC-1))]
        #R_con = blkdig([R_con kron(R_mat, Matrix{Float64}(I,r[i],r[i]))])
        #q_con = [q_con zeros(r[i]*size(R_mat,1),1)]
    #end
    
    # Indicator for m/q blocks in observation matrix
    bl_idxM = convert.(Bool, bl)
    bl_idxQ = convert.(Bool, repeat(bl, 1, ppC))
    R_con = kron(R_mat, Matrix{Float64}(I, ppC,ppC))
    q_con = repeat(q, ppC)
    
    i_idio_M = i_idio[1:nM]
    n_idio_M = length(findall(i_idio_M))
    #c_i_idio = cumsum(i_idio, [1])
    
    for i in 1:n_bl # loop through unique loading
        #println(i)
        bl_i = bl[i,:]
        rs = sum(convert.(Bool, bl_i)) # total num of blocks loaded (e.g [1 0 0 1] = 2)
        idx_i = ismember(blocks, bl_i) # indices for [1 0 0 1]
        idx_iM = idx_i[idx_i .< nM+1] # consider only monthly variables
        n_i = length(idx_iM)

        denom = zeros(n_i*rs, n_i*rs)
        nom = zeros(n_i, rs)
        
        ### for input robustness
        i_idio_ii = i_idio_M[idx_iM]
        #i_idio_ii = c_i_idio(idx_iM)
        #i_idio_ii = i_idio_ii(i_idio_i)
        global bl_idxM_i = findall(bl_idxM[i,:])
        
        #println("before update monthly variable")
        ## update monthly variables
        for t in 1:T
            Wt = Diagonal(.!(nanY[idx_iM,t]))
            denom = denom + kron(x_sm[bl_idxM_i, t+1]*x_sm[bl_idxM_i, t+1]' 
                + σ_sm[bl_idxM_i, bl_idxM_i, t+1], Wt)
            
            nom = nom + y[idx_iM,t]*x_sm[bl_idxM_i,t+1]' - Wt[:, i_idio_ii]*(x_sm[i_idio_ii.+rp1, t+1]*x_sm[bl_idxM_i,t+1]'
                + σ_sm[i_idio_ii.+rp1, bl_idxM_i, t+1])
        end
        vec_G = inv(denom)*nom[:] #Eq13 in BGR 2010
        println(size(reshape(vec_G, n_i, rs)))
        G_new[idx_iM, bl_idxM_i] = reshape(vec_G, n_i, rs)
        #println("successfully update monthly variable")
        ## update quarterly variables
        
        #println("before update quarterly variable")
        idx_iQ = idx_i[idx_i .> nM] # consider only quarterly variables
        global bl_idxQ_i = findall(bl_idxQ[i,:])
        global bl_idxQ_ii = findall(bl_idxQ[i, 1:end-ppC])
        rps = rs*ppC
        R_con_i = R_con[bl_idxQ_ii, bl_idxQ_i]
        q_con_i = q_con[bl_idxQ_ii]
        #no_c = .!(any(R_con_i,2))
        #R_con_i[no_c, :] = []
        #q_con_i[no_c, :] = []
        
        for j in idx_iQ
            denom = zeros(rps,rps)
            nom = zeros(1, rps)
            
            idx_jQ = j - nM
            global i_idio_jQ = (rp1+n_idio_M+5*(idx_jQ-1)+1:rp1+n_idio_M + 5*idx_jQ)

            
            σ_0_new[i_idio_jQ, i_idio_jQ] = Diagonal(σ_sm[i_idio_jQ, i_idio_jQ, 1])
            A_new[i_idio_jQ[1], i_idio_jQ[1]] = A_i[i_idio_jQ[1]-rp1, i_idio_jQ[1]-rp1]
            Q_new[i_idio_jQ[1], i_idio_jQ[1]] = Q_i[i_idio_jQ[1]-rp1, i_idio_jQ[1]-rp1]
            
            for t in 1:T
                Wt = Diagonal([.!(nanY[j,t])])
                denom = denom+kron(x_sm[bl_idxQ_i, t+1]*x_sm[bl_idxQ_i, t+1]' + σ_sm[bl_idxQ_i, bl_idxQ_i, t+1], Wt)
                nom = nom +y[j,t]*x_sm[bl_idxQ_i, t+1]'
                nom = nom - Wt*([1 2 3 2 1]*x_sm[i_idio_jQ,t+1]*x_sm[bl_idxQ_i,t+1]'+[1 2 3 2 1]*σ_sm[i_idio_jQ, bl_idxQ_i,t+1])
            end
            
            G_i = inv(denom)*nom'
            #BGR eq13
            G_i_constr = G_i - inv(denom)*R_con_i'*inv(R_con_i*denom*R_con_i')*(R_con_i*G_i -q_con_i)
            #println(size(G_i_constr))
            G_new[j,bl_idxQ_i] = G_i_constr
        end
        #println("after update quarterly variable")
    end
    #println("Observation Equation DONE")
    
    ### Update covariance of residuals for observation equation
    #R_new = zeros(n,n)
    #for t in 1:T
    #    Wt = Diagonal(.!(nanY[:,t]))
    #    R_new = R_new + (y[:,t] - Wt*G_new*x_sm[:,t+1])*(y[:,t]-Wt*G_new*x_sm[:,t+1])'+Wt*G_new*σ_sm[:,:,t+1]*G_new'*Wt + (Matrix{Float64}(I,n,n)-Wt)*R*(Matrix{Float64}(I,n,n)-Wt)
    #end
    
    #R_new = R_new / T
    #RR = Diagonal(R_new)
    #RR[i_idio+_M] = 1e-04
    #RR[nM+1:end] = 1e-04
    #R_new = Diagonal(RR)
    R_new = R #### There is no need to update R according to the way they set up state-space representation
    
    return G_new, R_new, A_new, Q_new, x_0, σ_0_new, loglik
end

EMstep (generic function with 1 method)

In [85]:
### Y_t = C_t Z_t + e_t for e_t ~ N(0, R)
### Z_t = A Z_{t-1} + mu_t for mu_t ~ N(0,Q)
function runKF(Y, A, C, Q, R, Z_0, V_0)
    S = SKF(Y,A,C,Q,R,Z_0,V_0)
    S, S1 = FIS(A,S)
    
    zsmooth = S1.ZmT
    Vsmooth = S1.VmT
    VVsmooth = S1.VmT_1
    loglik = S.loglik
    
    println("Successfully run Kalman Filter Algorithm")
    println(S.loglik)
    return zsmooth, Vsmooth, VVsmooth, loglik
end

function SKF(Y, A, C, Q, R, Z_0, V_0)
    # Initialize output values
    m = size(C,2)
    nobs = size(Y,2) # number of observations
    S = Sm(zeros(m, nobs), zeros(m, nobs+1), zeros(m,m, nobs), zeros(m,m, nobs+1), 0, zeros(m,m))
    
    Zu = Z_0
    Vu = V_0
    
    print(Z_0)
    
    S.ZmU[:,1] = Zu
    S.VmU[:,:,1] = Vu
    
    print("enter filtering procedure")
    # Kalman filter procedure
    for t in 1:nobs
        Z = A*Zu
        V = A*Vu*A' +Q
        #V = 0.5*(V+V')
        
        ### allow us to access C_t, VCF outside loop
        global C_t, VCF
        
        (Y_t, C_t, R_t) = MissData(Y[:,t], C, R)
        
        if isempty(Y_t)
            Zu = Z
            Vu = V
        else
            VC = V*C_t'
            temp = C_t*VC +R_t
            iF = inv(temp)
            
            VCF = VC*iF
            innov = Y_t - C_t*Z
            Zu = Z + VCF*innov
            
            Vu = V - VCF*C_t*V
            #Vu = 0.5*(Vu+Vu')
            
            #S.loglik = S.loglik - 0.5*(logdet(temp) + (innov'*iF*innov)[1])
        end
        
        S.loglik = 0
        
        ### Store output
        S.Zm[:,t] = Z
        S.Vm[:,:,t] = V
        S.ZmU[:,t+1] = Zu
        S.VmU[:,:,t+1] = Vu
    end
    
    #if isempty(Y_t)
    #    S.k_t = zeros(m,m)
    #else
    #    S.k_t = VCF*C_t
    #end
    
    ### Store Kalman gain
    S.k_t = VCF*C_t
    return S
end

### Fixed Interval Smoothing
function FIS(A,S)
    (m, nobs) = size(S.Zm)
    S1 = Sm1(zeros(m, nobs+1), zeros(m,m,nobs+1), zeros(m,m,nobs))
    
    S1.ZmT[:, nobs+1] = S.ZmU[:,nobs+1]
    S1.VmT[:,:,nobs+1] = S.VmU[:,:,nobs+1]
    S1.VmT_1[:,:,nobs] = (Matrix{Float64}(I, m, m)-S.k_t)*A*(S.VmU[:,:,nobs])
    
    J_2 = S.VmU[:,:,nobs]*A'*pinv(S.Vm[:,:,nobs])
    
    # Run smoothing algorithm
    for t in nobs:-1:1
        VmU = S.VmU[:,:,t]
        Vm1 = S.Vm[:,:,t]
        V_T = S1.VmT[:,:,t+1]
        V_T1 = S1.VmT_1[:,:,t]
        J_1 = J_2
        
        # update smoothed factor estimate / covariance matrix
        S1.ZmT[:,t] = S.ZmU[:,t]+J_1*(S1.ZmT[:,t+1] - A*S.ZmU[:,t])
        S1.VmT[:,:,t] = VmU + J_1*(V_T - Vm1)*J_1'
        
        if t > 1
            J_2 = S.VmU[:,:,t-1]*A'*pinv(S.Vm[:,:,t-1])
            S1.VmT_1[:,:,t-1] = VmU*J_2' + J_1*(V_T1 - A*VmU)*J_2'
        end
    end
    
    return S, S1
end

function MissData(y,C,R)
    ix = .!ismissing.(y)
    e = Matrix{Float64}(I,size(y,1),size(y,1))
    L = e[:,ix]
    y = y[ix]
    C = C[ix,:]
    R = R[ix,ix]
    
    return y, C, R
end

#function InitialCond -- PCA for prior information

function em_converged(loglik, previous_loglik, threshold = 1e-4, check_decreased = 1)
    converged = 0
    decrease = 0
    
    if check_decreased
        if loglik - previous_loglik < -1e-3
            printf(1, "******likelihood decreased from %6.4f to %6.4f!\n", previous_loglik, loglik)
            decrease = 1
        end
    end
    
    delta_loglik = abs(loglik - previous_loglik)
    avg_loglik = (abs(loglik) + abs(previous_loglik) + eps)/2
    
    if (delta_loglik / avg_loglik) < threshold
        converged = 1
    end
    
    println("converged")
    print(converged)
    
    return converged, decrease 
end

em_converged (generic function with 3 methods)

In [86]:
A_θ = convert(Matrix{Float64}, CSV.read("A_theta.csv", header = false))
G_θ = convert(Matrix{Float64}, CSV.read("G_theta.csv", header = false))
Q_θ = convert(Matrix{Float64}, CSV.read("Q_theta.csv", header = false))
R_θ = convert(Matrix{Float64}, CSV.read("R_theta.csv", header = false))
Z_θ = convert(Matrix{Float64}, CSV.read("Z_theta.csv", header = false))
V_θ = convert(Matrix{Float64}, CSV.read("V_theta.csv", header = false))
block = convert(Matrix{Float64}, CSV.read("block.csv", header = false))
X = CSV.read("X.csv", header = true)
X = convert(Matrix{Union{Missing,Float64}},X[2:end,])
print("Success")

Success

In [87]:
### Trial
Sp = Spec(1,1,2,block)
Inits = inits(A_θ, G_θ, Q_θ, R_θ, Z_θ, V_θ)

x_sm = dfm(X', Sp, Inits)


run EM loop
1
[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0]enter filtering procedureSuccessfully run Kalman Filter Algorithm
0.0
(47, 2)
(40, 2)
(28, 2)
(3, 2)
2
[0.0249923, -2.3395e-7, 0.000113481, 3.97244e-6, -0.0764708, -1.16817e-5, 9.16626e-13, 1.79917e-12, -1.71548e-11, 3.59

[-0.0165446, 0.212023, -0.0124946, 0.0207844, -0.961251, -0.202079, -6.9441e-10, 4.7787e-10, 2.6505e-11, -2.05664, -0.0481416, -2.31008e-12, 4.31068e-11, -1.44608e-10, -1.98846, 0.532872, 0.0, 0.0, 0.0, 0.787073, 0.0, 0.0, 0.0, 0.0, 0.0, 9.67921e-5, 0.00119738, -0.0210248, 5.20286e-6, 0.000277519, 0.0511625, 0.0326757, 0.0068999, -0.010809, -0.00188315, -0.0113239, 0.0238343, 0.0867216, 0.0826062, 0.0112158, 0.0473979, 0.00411566, 0.0625777, 0.858136, 0.0273626, 0.00305517, 0.329436, 0.00866164, 0.0257942, 0.000451714, -0.670993, 0.470959, 0.89207, 0.00393871, 0.00611491, -0.0232207, -0.0148159, 0.011676, 0.0181158, 0.0069434, 0.00376009, 0.0059784, 0.00291299, 0.013221, -0.00164988, 0.00408216, -0.0091558, 0.0471715, -0.00344573, 0.00444452, 0.00194315, -0.11678, -0.152391, -0.0306468, -0.00179924, -0.00211695, 0.000645551, -0.00459072, 0.000982759, -0.0035285, -0.00222508, -0.000535059, 0.000148114, 0.000367748, 0.0104691, 0.002422, -0.00391462, 0.00483137, 0.0498511, -0.109594, -0.0

120×724 Array{Float64,2}:
 -8.67503e34  -5.6705e34   -1.31281e36  …  -1.06352e14   2.40393e13
 -8.69458e34  -5.69755e34  -1.30913e36     -1.06358e14   2.40401e13
 -8.65387e34  -5.63122e34  -1.30651e36     -1.06364e14   2.40407e13
 -8.70346e34  -5.74909e34  -1.31315e36     -1.0636e14    2.40413e13
 -3.19159e37  -6.04998e37   5.9609e37       5.66835e13   2.80694e13
 -8.64472e34  -5.62654e34  -1.30965e36  …  -1.06365e14   2.40412e13
 -8.62187e34  -5.57155e34  -1.31121e36     -1.06363e14   2.40406e13
 -8.59714e34  -5.50868e34  -1.31146e36     -1.06369e14   2.40412e13
 -8.62153e34  -5.57258e34  -1.30905e36     -1.06368e14   2.40405e13
 -8.71968e34  -5.75347e34  -1.30929e36     -1.06367e14   2.40421e13
 -8.59908e34  -5.52431e34  -1.31158e36  …  -1.06359e14   2.40393e13
 -8.54557e34  -5.42304e34  -1.3137e36      -1.06371e14   2.40408e13
 -8.68443e34  -5.70489e34  -1.3122e36      -1.06361e14   2.40409e13
  ⋮                                     ⋱                          
 -3.19223e37  -6.05118

In [80]:
A_θ

153×153 Array{Float64,2}:
 0.996564   0.0        0.0         …  0.0   0.0       0.0  0.0  0.0  0.0
 0.0       -0.0792658  0.0            0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.00164138     0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.0            0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.0            0.0   0.0       0.0  0.0  0.0  0.0
 1.0        0.0        0.0         …  0.0   0.0       0.0  0.0  0.0  0.0
 0.0        1.0        0.0            0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        1.0            0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.0            0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.0            0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.0         …  0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.0            0.0   0.0       0.0  0.0  0.0  0.0
 0.0        0.0        0.0            0.0   0.0       0.0  0.0  0.0  0.0
 ⋮                       

In [81]:
G_θ

120×153 Array{Float64,2}:
     0.00861347     0.0         0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0113917      0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.00549878     0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.012622       0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
   460.984          0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.00414605     0.0         0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.000841572    0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
    -0.0027584      0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.000777891    0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0149897      0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
    -0.00246184     0.0         0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
    -0.0102211      0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.00988565     0.0         0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     ⋮      

In [82]:
Z_θ

153×1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [94]:
x_sm'

724×120 Adjoint{Float64,Array{Float64,2}}:
 -8.67503e34  -8.69458e34  -8.65387e34  …   1.71697e51  -5.06247e49
 -5.6705e34   -5.69755e34  -5.63122e34     -8.05676e50   2.37553e49
 -1.31281e36  -1.30913e36  -1.30651e36      9.03395e50  -2.66366e49
  1.35611e37   1.35286e37   1.34821e37     -1.10098e51   3.24623e49
 -3.59079e35  -3.57877e35  -3.5721e35       5.55624e48  -1.63825e47
 -2.60635e37  -2.59967e37  -2.59672e37  …   1.05065e50  -3.09783e48
 -1.73453e37  -1.73418e37  -1.73562e37      7.39246e49  -2.17966e48
 -1.92705e38  -1.92719e38  -1.92848e38      8.58402e50  -2.53099e49
  1.39828e38   1.40534e38   1.40884e38     -4.01118e50   1.18269e49
 -4.3931e38   -4.39879e38  -4.4018e38       1.58208e51  -4.66476e49
 -3.1919e37   -3.19337e37  -3.15679e37  …   5.75892e50  -1.69801e49
  2.77906e37   2.77949e37   2.77687e37     -1.55201e49   4.57609e47
 -8.58227e38  -8.59507e38  -8.58737e38      3.75642e51  -1.10758e50
  ⋮                                     ⋱                          
  1.8

In [92]:
X

724×120 Array{Union{Missing, Float64},2}:
  0.0012526     0.0010653     0.00767048   …   0.0148145     0.00144012 
  0.0037502     0.00486275    0.00671482       0.0           0.0        
  0.00381723    0.00450843   -0.0062556        0.0           0.0        
  0.00314537    0.00411852    0.00930512      -0.00678716   -0.000394184
  0.000427094   0.000511156   0.000966653      0.0           0.0        
 -0.00326644   -0.0033188    -0.00604413   …   0.0           0.0        
 -0.00837617   -0.00822697    0.00331371      -0.00465258   -0.00202109 
 -0.00189259   -0.00255374    0.007322         0.0           0.0        
 -0.00145783   -0.00131382   -0.00950251       0.0           0.0        
  0.00491526    0.00417518   -0.00312461       0.0147253     0.00167232 
  0.00751591    0.00884201   -0.00379359   …   0.0           0.0        
  0.00051197    0.00213642    0.000104457      0.0           0.0        
 -0.00153613   -0.00159242    0.00164789      -0.0129074     0.000754914
  ⋮      

In [155]:
abs(-1)

1

In [192]:
Diagonal(Diagonal([1 2; 3 4]))

2×2 Diagonal{Int64,Array{Int64,1}}:
 1  ⋅
 ⋅  4